In [ ]:
def direction_cosines(ra, dec, ra0, dec0):
    dra = ra - ra0
    l = np.cos(dec) * np.sin(dra)
    m = np.sin(dec) * np.cos(dec0) - np.cos(dec) * np.sin(dec0) * np.cos(dra)
    n = np.sqrt(1.0 - l**2 - m**2)     # o: n = sin dec * sin dec0 + cos dec * cos dec0 * cos dra
    return l, m, n


delta0 = delta_src
ra0 = ra_src

l0, m0, n0 = direction_cosines(ra_src, delta_src, ra0, delta0)


S0 = 1.0
# ejemplo de primary beam gaussiana con FWHM ~ 1 deg (ajusta según tu antena)
# sigma_rad = FWHM / (2*sqrt(2*ln2))
fwhm_deg = 1.0
sigma_rad = np.deg2rad(fwhm_deg) / (2*np.sqrt(2*np.log(2)))
A_lm = np.exp(- (l0**2 + m0**2) / (2 * sigma_rad**2))  # si quieres A=1, pon A_lm = 1.0

# --- 5) uvw: forma (702, 145, 3) -> ejemplo: convertir a (N,) para facilidad ---
# uvw must be in wavelengths already; si está en metros, conviértelo con uvw * freq / c
u = uvw_lambda[...,0].ravel()
v = uvw_lambda[...,1].ravel()
w = uvw_lambda[...,2].ravel()

# --- 6) Calcular visibilidades para la fuente puntual ---
phase = 2*np.pi*(u*l0 + v*m0 + w*(n0 - 1.0))
V = A_lm * S0 / n0 * np.exp(1j * phase)   # resultado complejo, shape (702*145,)

# Si quieres volver a la forma (702,145):
V = V.reshape(uvw_lambda.shape[0], uvw_lambda.shape[1])

print("l0,m0,n0:", l0, m0, n0)
print("Visibilidad shape:", V.shape)

print(V)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- 1) Coordenadas de Sirius (J2000) ---
ra_h = 6 + 45/60 + 8.9/3600
ra0 = ra_h * (2*np.pi/24)
dec_deg = -16 - 42/60 - 58/3600
dec0 = np.deg2rad(dec_deg)

# --- 2) Función para calcular (l,m,n) ---
def calc_lmn(ra, dec, ra0, dec0):
    dra = ra - ra0
    l = np.cos(dec) * np.sin(dra)
    m = np.sin(dec) * np.cos(dec0) - np.cos(dec) * np.sin(dec0) * np.cos(dra)
    n = np.sin(dec) * np.sin(dec0) + np.cos(dec) * np.cos(dec0) * np.cos(dra)
    return l, m, n

# --- 3) Definimos las dos fuentes ---
# Fuente 1: centro (Sirius)
sources = []
S1 = 1.0
l1, m1, n1 = calc_lmn(ra0, dec0, ra0, dec0)
sources.append((S1, l1, m1, n1))

# Fuente 2: desplazada +1 grado en RA
delta_ra = np.deg2rad(1.0)
S2 = 0.8
l2, m2, n2 = calc_lmn(ra0 + delta_ra, dec0, ra0, dec0)
sources.append((S2, l2, m2, n2))

# --- 4) Primary beam (asumimos A=1) ---
A = 1.0

# --- 5) Extraemos u,v,w ---
u = uvw[...,0].ravel()
v = uvw[...,1].ravel()
w = uvw[...,2].ravel()

# --- 6) Calculamos visibilidades totales ---
V_total = np.zeros_like(u, dtype=complex)
for S, l, m, n in sources:
    V_total += A * S / n * np.exp(2j * np.pi * (u*l + v*m + w*(n - 1)))

# Restauramos forma original (702,145)
V_total = V_total.reshape(uvw.shape[0], uvw.shape[1])

print(V_total)

# --- 7) Visualizamos amplitud y fase ---
amp = np.abs(V_total)
phase = np.angle(V_total)

plt.figure(figsize=(8,4))
plt.imshow(amp, aspect='auto', origin='lower')
plt.colorbar(label='Amplitude')
plt.title('Amplitude of V (2 sources)')
plt.xlabel('frequency index')
plt.ylabel('baseline index')

plt.figure(figsize=(8,4))
plt.imshow(phase, aspect='auto', origin='lower', cmap='twilight')
plt.colorbar(label='Phase [rad]')
plt.title('Phase of V (2 sources)')
plt.xlabel('frequency index')
plt.ylabel('baseline index')
plt.show()

In [ ]:
def direction_cosines(ra_rad, dec_rad, ra0_rad, dec0_rad):
    """
    ra_rad, dec_rad: arrays (Nsrc,) or scalars, in radians
    ra0_rad, dec0_rad: scalars (pointing) in radians
    returns l, m, n arrays same shape as ra_rad
    """
    dalpha = ra_rad - ra0_rad
    cosd = np.cos(dec_rad)
    sind = np.sin(dec_rad)
    cosd0 = np.cos(dec0_rad)
    sind0 = np.sin(dec0_rad)

    l = cosd * np.sin(dalpha)
    m = sind * cosd0 - cosd * sind0 * np.cos(dalpha)
    n = sind * sind0 + cosd * cosd0 * np.cos(dalpha)
    return l, m, n

# ---------------- generar visibilidades ----------------
def visibilities_from_sources(uvw_lambda, sources, ra0_deg, dec0_deg, sigma_pb=0.05, freq_hz=None):
    """
    uvw_lambda: (Nvis,3) u,v,w in lambda units
    sources: list of dicts [{'ra_deg':..., 'dec_deg':..., 'S0':...}, ...]
    ra0_deg, dec0_deg: pointing in degrees
    sigma_pb: primary beam sigma (radians)
    returns V_total (Nvis,) complex
    """
    # u,v,w arrays
    u = uvw_lambda[:,0]
    v = uvw_lambda[:,1]
    w = uvw_lambda[:,2]

    # pointing in radians
    ra0 = np.deg2rad(ra0_deg)
    dec0 = np.deg2rad(dec0_deg)

    # Prepare source arrays
    ras = np.array([src['ra_deg'] for src in sources])
    decs = np.array([src['dec_deg'] for src in sources])
    S0s  = np.array([src.get('S0', 1.0) for src in sources])

    ras_rad = np.deg2rad(ras)
    decs_rad = np.deg2rad(decs)

    # Get l,m,n for all sources
    l_src, m_src, n_src = direction_cosines(ras_rad, decs_rad, ra0, dec0)  # shape (Nsrc,)

    # Primary beam (evaluated at source position) -> gaussian circular
    A_src = np.exp(-(l_src**2 + m_src**2) / (2 * sigma_pb**2))  # shape (Nsrc,)

    # For each source compute its visibility vector (vectorized over visibilities)
    # We'll sum contributions: V_total = sum_s A_s * S_s / n_s * exp(2π i (u l_s + v m_s + w (n_s-1)))
    V_total = np.zeros(u.shape, dtype=complex)

    # Loop over sources (Nsrc usually small; loop is fine). If Nsrc large, vectorizar más.
    two_pi_i = 2j * np.pi
    for ls, ms, ns, As, Ss in zip(l_src, m_src, n_src, A_src, S0s):
        phase = two_pi_i * (u * ls + v * ms + w * (ns - 1.0))
        V_s = As * Ss / ns * np.exp(phase)
        V_total += V_s

    # weights omega = 1 (según enunciado)
    omega = np.ones_like(V_total, dtype=float)
    return V_total, omega, l_src, m_src, n_src

# ---------------- ejemplo de uso ----------------
# Supón que uvw_lambda ya está definido (Nvis,3)
# Ejemplo de fuentes: 2 fuentes, una en el centro, otra 0.5 deg al este



sources = [
    {"ra_deg": 101.2875, "dec_deg": -16.7161, "S0": np.random.uniform(0.5, 1.5)},            # Sirius (centro)
    {"ra_deg": 101.2875 + 0.5, "dec_deg": -16.7161, "S0": np.random.uniform(0.2, 1.0)}       # 0.5° Este
]

# pointing (puede ser la RA/Dec de Sirius)
ra0_deg = 101.2875
dec0_deg = -16.7161

# calcular visibilidades
V, omega, l_src, m_src, n_src = visibilities_from_sources(uvw_lambda, sources, ra0_deg, dec0_deg)

# ahora V es un vector de complejos que podes gridear con la rutina que ya tienes

amp = np.log1p(np.abs(V))
phase = np.angle(V)

plt.figure(figsize=(8,4))
plt.imshow(amp, aspect='auto', origin='lower')
plt.colorbar(label='Amplitude')
plt.title('Amplitude of V (2 sources)')
plt.xlabel('frequency index')
plt.ylabel('baseline index')

plt.figure(figsize=(8,4))
plt.imshow(phase, aspect='auto', origin='lower', cmap='twilight')
plt.colorbar(label='Phase [rad]')
plt.title('Phase of V (2 sources)')
plt.xlabel('frequency index')
plt.ylabel('baseline index')
plt.show()


In [ ]:
import numpy as np

def generate_random_sources(ra0_deg, dec0_deg, N=50, max_offset_deg=1.0, flux_range=(0.1, 1.0), seed=None):
    """
    Genera N fuentes puntuales alrededor del centro (ra0, dec0)
    en un radio máximo de max_offset_deg.
    """
    rng = np.random.default_rng(seed)
    ras = ra0_deg + rng.uniform(-max_offset_deg, max_offset_deg, N)
    decs = dec0_deg + rng.uniform(-max_offset_deg, max_offset_deg, N)
    fluxes = rng.uniform(flux_range[0], flux_range[1], N)

    sources = [{"ra_deg": ra, "dec_deg": dec, "S0": S} for ra, dec, S in zip(ras, decs, fluxes)]
    return sources

# --- Ejemplo de uso ---
ra0_deg = 101.2875   # Sirius A
dec0_deg = -16.7161
sources = generate_random_sources(ra0_deg, dec0_deg, N=200, max_offset_deg=1.0, flux_range=(0.2, 1.0), seed=42)

# Ahora puedes pasarlas directo a tu función:
V, omega, l_src, m_src, n_src = visibilities_from_sources(uvw_lambda, sources, ra0_deg, dec0_deg)

print(f"Simuladas {len(sources)} fuentes puntuales.")

amp = np.log1p(np.abs(V))
phase = np.angle(V)

plt.figure(figsize=(8,4))
plt.imshow(amp, aspect='auto', origin='lower')
plt.colorbar(label='Amplitude')
plt.title('Amplitude of V (2 sources)')
plt.xlabel('frequency index')
plt.ylabel('baseline index')

plt.figure(figsize=(8,4))
plt.imshow(phase, aspect='auto', origin='lower', cmap='twilight')
plt.colorbar(label='Phase [rad]')
plt.title('Phase of V (2 sources)')
plt.xlabel('frequency index')
plt.ylabel('baseline index')
plt.show()


plt.figure(figsize=(6,6))
plt.scatter([s["ra_deg"] for s in sources], [s["dec_deg"] for s in sources],
            c=[s["S0"] for s in sources], cmap="plasma", s=50)
plt.gca().invert_xaxis()  # RA aumenta hacia la izquierda en astronomía
plt.xlabel("RA [deg]")
plt.ylabel("Dec [deg]")
plt.title("Fuentes puntuales simuladas")
plt.colorbar(label="Flujo S₀")
plt.show()


In [ ]:

# Aseguramos que u, v, V sean vectores 1D
u = uvw_lambda[:,0].flatten()
v = uvw_lambda[:,1].flatten()
V = V.flatten()

# Tamaño de grilla
N = 512
u_max = np.max(np.abs(u))
v_max = np.max(np.abs(v))

u_grid = np.linspace(-u_max, u_max, N)
v_grid = np.linspace(-v_max, v_max, N)

V_grid = np.zeros((N, N), dtype=complex)
W_grid = np.zeros((N, N))

du = u_grid[1] - u_grid[0]
dv = v_grid[1] - v_grid[0]

for i in range(len(V)):
    iu = int(np.round((u[i] - (-u_max)) / du))
    iv = int(np.round((v[i] - (-v_max)) / dv))

    if 0 <= iu < N and 0 <= iv < N:
        V_grid[iv, iu] += V[i]
        W_grid[iv, iu] += 1

# Normalizamos los píxeles con visibilidades
mask = W_grid > 0
V_grid[mask] /= W_grid[mask]


V_grid_sym = V_grid + np.conj(np.flip(np.flip(V_grid, axis=0), axis=1))
dirty_image = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(V_grid_sym)))
dirty_image = np.real(dirty_image)


plt.figure(figsize=(6,5))
plt.imshow(dirty_image, cmap='inferno', origin='lower', extent=[-1,1,-1,1])
plt.colorbar(label='Intensidad')
plt.xlabel('l (rad aprox)')
plt.ylabel('m (rad aprox)')
plt.title('Dirty Image')
plt.show()


In [ ]:
from PIL import Image

def grayscale_and_normlize(route):
  img = Image.open(route)
  gray_img = img.convert('L')
  normalized_img = np.array(gray_img) / 255.0

  plt.imshow(normalized_img, cmap='gray')
  plt.axis('off')
  plt.show()

  return normalized_img


def img_to_fourier(img):
  f2 = np.fft.ifftshift(img)
  f2 = np.fft.fft2(f2)
  f2_shifted = np.fft.fftshift(f2)

  return f2, f2_shifted

def plot_mag_phase(img, cmap="twilight"):

    f2, f2_shifted = img_to_fourier(img)

    # Magnitudes y fases para f2 (no desplazado)
    magnitude = np.log1p(np.abs(f2))
    phase = np.angle(f2)

    # Magnitudes y fases para f2_shifted (desplazado)
    magnitude_shifted = np.log1p(np.abs(f2_shifted))
    phase_shifted = np.angle(f2_shifted)

    fig, ax = plt.subplots(2, 2, figsize=(10, 5))

    # Magnitud
    im1 = ax[0][0].imshow(magnitude, cmap=cmap)
    ax[0][0].set_title("Espectro de Magnitud (FFT2)")
    fig.colorbar(im1, ax=ax[0][0], fraction=0.046, pad=0.04)

    im2 = ax[0][1].imshow(magnitude_shifted, cmap=cmap)
    ax[0][1].set_title("Espectro de Magnitud (FFT2) shifted")
    fig.colorbar(im2, ax=ax[0][1], fraction=0.046, pad=0.04)

    # Fase
    im3 = ax[1][0].imshow(phase, cmap=cmap, vmin=-np.pi, vmax=np.pi)
    ax[1][0].set_title("Espectro de Fase (FFT2)")
    fig.colorbar(im3, ax=ax[1][0], fraction=0.046, pad=0.04)

    im4 = ax[1][1].imshow(phase_shifted, cmap=cmap, vmin=-np.pi, vmax=np.pi)
    ax[1][1].set_title("Espectro de Fase (FFT2) shifted")
    fig.colorbar(im4, ax=ax[1][1], fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()

img = grayscale_and_normlize("../antenna_arrays/image.jpg")

plot_mag_phase(img)



In [1]:
from scipy.interpolate import griddata

def grid_visibilities(V, uvw_lambda, Npix=256):
    """
    Gridding simple de visibilidades (u,v) en una grilla regular 2D.
    """

    print("uvw_lambda shape:", uvw_lambda.shape)
    print("V shape:", V.shape)

    
    u = uvw_lambda[:,0]
    v = uvw_lambda[:,1]
    V_flat = V.flatten()
    print(V_flat.shape)

    u_max = np.max(np.abs(u))
    v_max = np.max(np.abs(v))

    # Grilla regular
    u_grid = np.linspace(-u_max, u_max, Npix)
    v_grid = np.linspace(-v_max, v_max, Npix)
    U, Vv = np.meshgrid(u_grid, v_grid)

    # Interpolación: griddata asigna valores complejos a la grilla
    V_grid = griddata((u, v), V_flat, (U, Vv), method='nearest', fill_value=0.0+0.0j)
    return V_grid, u_grid, v_grid


# Aplanar uvw_lambda en (702*145, 3)
uvw_flat = uvw_lambda.reshape(-1, 3)

# Tomar sólo u y v
u = uvw_flat[:, 0]
v = uvw_flat[:, 1]

# Expandir visibilidades a la misma cantidad de puntos
# Si tus V tienen 3 frecuencias, repetimos cada baseline 145 veces
V_expanded = np.repeat(V, 145, axis=0)

# Si solo quieres una frecuencia (por ejemplo, la primera):
V_flat = V_expanded[:, -1]  # o promedio en caso de querer una sola imagen

V_grid, u_grid, v_grid = grid_visibilities(V_flat, uvw_flat, Npix=256)

from numpy.fft import ifftshift, fftshift, ifft2

# Transformada inversa
image = fftshift(ifft2(ifftshift(V_grid)))
intensity = np.log1p(np.abs(image))

plt.figure(figsize=(6,6))
plt.imshow(intensity, origin='lower', cmap='viridis',
           extent=[-1,1,-1,1])
plt.xlabel('l [rad aprox]')
plt.ylabel('m [rad aprox]')
plt.title('Imagen reconstruida del cielo simulado')
plt.colorbar(label='Intensidad')
plt.show()


NameError: name 'uvw_lambda' is not defined